In [2]:
import pandas as pd
import numpy as np
import os
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
path = "data"
df_i = pd.read_pickle(os.path.join(path, "df_i_fe.pkl"))
df_c_train = pd.read_pickle(os.path.join(path, "df_c_train_fe.pkl"))
df_t_train = pd.read_pickle(os.path.join(path, "df_t_train_fe.pkl"))

In [4]:
df_c_train.head()

,customer_id,total_spent,spending_power,preferred_products,preferred_colors,preferred_perceived_colors,combined_preferences,combined_preferences_embeddings,avg_time_diff_btw_purchase,hours_between_purchases
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0.111814,Medium,"Garment Upper body, Garment Full body","Black, Light Pink",Dark,"Garment Upper body, Garment Full body, Black, ...","[-0.15802608, 0.010872153, 0.14405754, 0.05493...",11 days 12:00:00,276.0
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.782712,High,"Garment Upper body, Garment Lower body","Black, Greenish Khaki",Dark,"Garment Upper body, Garment Lower body, Black,...","[-0.17723349, -0.13966687, 0.16963258, 0.16061...",3 days 10:06:18.947368421,82.0
2,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0.060983,Medium,Underwear,Black,Dark,"Underwear, Black, Dark","[0.17214382, 0.26306865, 0.38482118, 0.0213303...",0 days 00:00:00,8760.0
3,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,0.042339,Low,Swimwear,Dark Green,Medium Dusty,"Swimwear, Dark Green, Medium Dusty","[0.06293525, 0.28026408, 0.16526614, 0.1578645...",0 days 00:00:00,8760.0
4,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,0.772729,High,"Garment Upper body, Garment Lower body","Black, Dark Blue",Dark,"Garment Upper body, Garment Lower body, Black,...","[-0.07929248, -0.035167325, 0.21449153, 0.1658...",1 days 21:31:02.068965517,46.0


In [5]:
df_t_train.head()

,t_dat,customer_id,article_id,price,sales_channel_id,time_diff
9663224,2019-05-02,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,697138006,0.010153,2,NaT
10754876,2019-05-25,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601006,0.050831,2,23 days
10754877,2019-05-25,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601006,0.050831,2,0 days
8522926,2019-04-09,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,723469005,0.025407,2,NaT
8522927,2019-04-09,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,640174001,0.042356,2,0 days


In [6]:
df_t_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3738625 entries, 9663224 to 9526822
Data columns (total 6 columns):
 #   Column            Dtype          
---  ------            -----          
 0   t_dat             datetime64[ns] 
 1   customer_id       object         
 2   article_id        int64          
 3   price             float64        
 4   sales_channel_id  int64          
 5   time_diff         timedelta64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(1), timedelta64[ns](1)
memory usage: 199.7+ MB


In [7]:
df_i.head()

,article_id,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,index_group_name,garment_group_name,detail_desc,structured_desc,embedding
0,108775015,Strap top,253,Vest top,Garment Upper body,Solid,Black,Dark,Black,Ladieswear,Jersey Basic,Jersey top with narrow shoulder straps.,"Strap top, Vest top, Garment Upper body, Dark,...","[0.338782, 1.1482849, 0.5620432, -0.60869545, ..."
1,108775044,Strap top,253,Vest top,Garment Upper body,Solid,White,Light,White,Ladieswear,Jersey Basic,Jersey top with narrow shoulder straps.,"Strap top, Vest top, Garment Upper body, Light...","[0.21248032, 1.1400336, 0.30288446, -0.3514614..."
2,108775051,Strap top (1),253,Vest top,Garment Upper body,Stripe,Off White,Dusty Light,White,Ladieswear,Jersey Basic,Jersey top with narrow shoulder straps.,"Strap top (1), Vest top, Garment Upper body, D...","[0.37116015, 1.2416054, 0.51160425, 0.07794681..."
3,110065001,OP T-shirt (Idro),306,Bra,Underwear,Solid,Black,Dark,Black,Ladieswear,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...","OP T-shirt (Idro), Bra, Underwear, Dark, Black","[1.217555, 0.8623115, -0.44922385, 0.12923041,..."
4,110065002,OP T-shirt (Idro),306,Bra,Underwear,Solid,White,Light,White,Ladieswear,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...","OP T-shirt (Idro), Bra, Underwear, Light, White","[1.0551672, 0.8517027, -0.7824279, 0.4599598, ..."


# Content-Based Filtering

In [51]:
class UserProfiles:
    def __init__(self, transactions, items):
        # CONSTANTS
        self.TIME_MAX = transactions['t_dat'].max()
        self.DECAY_CONSTANT = 0.05

        self.transactions = transactions
        self.items = items
        self.interacted = transactions.groupby('customer_id')['article_id'].apply(list).to_dict() # maps a user to a list of their purchased items
        self.time_interacted = transactions.groupby('customer_id')['t_dat'].apply(list).to_dict() # maps a user to a list of timestamps of their purchases
        self.embeddings = np.array([np.array(x) for x in items['embedding']])
        

    def calculate_exponential_weight(self, time_interacted):
        return np.exp(-1 * self.DECAY_CONSTANT * (self.TIME_MAX - time_interacted).days)


    def get_current_interactions(self, userid):
        sample_item_desc = []
        interacted_items = self.interacted[userid]

        for item in interacted_items:
            idx = df_i[df_i['article_id'] == item].index[0]
            sample_item_desc.append(idx)

        return df_i.iloc[sample_item_desc, :]

    def get_recommendations(self, k, userid):
        interacted_items = self.interacted[userid]
        interacted_timestamp = self.time_interacted[userid]

        weights = []
        scores = []

        # For each interacted item by the user
        for i in range(0, len(interacted_items)):
            # Compute the weight to be applied (adjusted for decay)
            weights.append(self.calculate_exponential_weight(interacted_timestamp[i])) 

            # Perform search for similar products
            idx = df_i[df_i['article_id'] == interacted_items[i]].index[0]
            query_embedding = df_i.iloc[idx, -1].reshape(1, -1) # get query embedding
            sim_vector = cosine_similarity(query_embedding, self.embeddings).flatten()

            # Assign the equivalent items' scores to be 0 (prevent interacted items from being recommended again)
            sim_vector[sim_vector >= 1.0] = 0
            scores.append(sim_vector)

        # apply weight to each corresponding score
        final_ranking = np.dot(weights, scores) / len(interacted_items)

        # Get Top K results
        recommend_list = final_ranking.argsort()[::-1][:k]

        # returns the list of indexes of recommended items
        return recommend_list

train_userprofile = UserProfiles(df_t_train, df_i)

In [52]:
indexes = train_userprofile.get_recommendations(10, '002cb6b0ee98f590c120fb5f3c458a6588287db25901341cdb0f5e5a8cf8edcb')
df_i.iloc[indexes, :]

,article_id,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,index_group_name,garment_group_name,detail_desc,structured_desc,embedding
79168,796735002,Cleo shirt dress,258,Blouse,Garment Upper body,All over pattern,Light Pink,Dusty Light,Pink,Baby/Children,Dresses/Skirts girls,"Dress in woven fabric with a collar, button pl...","Cleo shirt dress, Blouse, Garment Upper body, ...","[0.41106093, 1.3852245, 0.84533745, 0.3870755,..."
84864,817095001,DIV Amy a-line dress,258,Blouse,Garment Upper body,Solid,Light Purple,Dusty Light,Lilac Purple,Divided,Unknown,"Short, A-line dress in a crinkled viscose weav...","DIV Amy a-line dress, Blouse, Garment Upper bo...","[0.44026008, 1.2236857, 0.73303884, 0.39462766..."
99563,885410001,LOGG Maya dress,258,Blouse,Garment Upper body,Solid,Light Yellow,Dusty Light,Yellow,Ladieswear,Blouses,Knee-length dress in an airy cotton and viscos...,"LOGG Maya dress, Blouse, Garment Upper body, D...","[0.45205835, 1.3512555, 0.8487045, 0.2709197, ..."
72327,767292002,Talia dress,258,Blouse,Garment Upper body,Solid,Light Pink,Dusty Light,Pink,Baby/Children,Dresses/Skirts girls,"Dress in softly draping, patterned chiffon wit...","Talia dress, Blouse, Garment Upper body, Dusty...","[0.43675375, 1.5090743, 0.89305985, 0.46109262..."
56266,713718001,ED Martini wrap dress,254,Top,Garment Upper body,All over pattern,Light Green,Dusty Light,Grey,Ladieswear,Blouses,"Short wrap dress in soft, patterned satin with...","ED Martini wrap dress, Top, Garment Upper body...","[0.6369229, 1.4274502, 0.75208473, 0.29882172,..."
60603,730124001,Kim set blouse,258,Blouse,Garment Upper body,All over pattern,Light Green,Dusty Light,Green,Divided,Blouses,Short blouse in a patterned crêpe weave with a...,"Kim set blouse, Blouse, Garment Upper body, Du...","[0.36200827, 1.1562608, 0.6695952, 0.06616842,..."
33899,640098002,Kylie Blouse,254,Top,Garment Upper body,Jacquard,Light Orange,Dusty Light,Orange,Ladieswear,Dresses Ladies,"Blouse in a jacquard weave with pleats, a V-ne...","Kylie Blouse, Top, Garment Upper body, Dusty L...","[0.60234606, 1.4156651, 0.70457137, 0.03873213..."
93826,859808003,Amy Top,258,Blouse,Garment Upper body,Solid,Light Pink,Dusty Light,Pink,Ladieswear,Blouses,Short-sleeved blouse in softly draping satin. ...,"Amy Top, Blouse, Garment Upper body, Dusty Lig...","[0.5673993, 1.4046069, 0.727437, 0.1253092, 0...."
59466,723542003,Dana dress,254,Top,Garment Upper body,Solid,Light Turquoise,Light,Turquoise,Baby/Children,Jersey Fancy,Short-sleeved dress in slub cotton jersey with...,"Dana dress, Top, Garment Upper body, Light, Li...","[0.58684176, 1.587176, 0.6897934, 0.24800825, ..."
6138,502393003,CS Bernadette frill top,258,Blouse,Garment Upper body,Lace,Light Orange,Dusty Light,Orange,Ladieswear,Blouses,"Lace blouse with short, flounce-trimmed sleeve...","CS Bernadette frill top, Blouse, Garment Upper...","[0.38430104, 1.2071345, 0.58307284, 0.01625692..."
